In [ ]:
import numpy as np
import pandas as pd
import re
from datetime import datetime
from dateutil import parser


In [ ]:
filename = './input.txt'

schedule = pd.DataFrame(columns=['hours', 'minutes', 'keyword', 'extra'])
rg = re.compile('\[(\d{4}-\d{2}-\d{2}) (\d{2}):(\d{2})] ([a-zA-Z]*) ([#\d+|a-zA-Z]*)',re.IGNORECASE|re.DOTALL)
with open(filename) as input_file:
    for line in input_file:
        match = rg.search(line)
        if match:
            date = match.group(1)
            hours = match.group(2)
            minutes = match.group(3)

            keyword = match.group(4)
            extra = match.group(5)
            row = {'date': date,
                   'hours': hours, 
                   'minutes': minutes, 
                   'keyword': keyword, 
                   'extra': extra}
            schedule = schedule.append(row, ignore_index=True)

Increment day for guards that arrived early:

In [ ]:
for index, guard in schedule[(schedule['keyword'] == 'Guard') & (schedule['hours'] == '23')].iterrows():
    schedule.loc[index]['date'] = (datetime.strptime(schedule.loc[index]['date'], '%Y-%m-%d').date() + timedelta(days=1)).strftime('%Y-%m-%d')

In [ ]:
all_dates = schedule['date'].unique()
timetable = pd.DataFrame(columns=['Date', 'ID', 'Minute'])
np.zeros((len(all_dates), 60), dtype=bool)
for date in all_dates:
    guard = schedule[(schedule['date'] == date) & (schedule['keyword'] == 'Guard')].iloc[0]['extra']
    events = schedule[(schedule['date'] == date) & (schedule['keyword'] != 'Guard')].sort_values('minutes')
    minutes = np.ones(60, dtype=bool)
    for index, event in events.iterrows():
        minutes[int(event['minutes']):] = ~minutes[int(event['minutes']):]
    
    row = {
            'Date': date,
            'ID': guard, 
            'Minute': minutes,
          }
    timetable = timetable.append(row, ignore_index=True)    

### Question A

In [ ]:
all_guards = schedule[schedule['keyword'] == 'Guard']['extra'].unique()
guard_info = pd.DataFrame(columns=['ID', 'awake'])
for guard in all_guards:
    guard_nights = schedule[(schedule['extra'] == guard)]['date']
    awake = [timetable[timetable['Date'] == night].iloc[0]['Minute'] for night in guard_nights]

    row = {
            'ID': guard, 
            'awake': sum(sum(awake) / len(guard_nights)),
          }
    guard_info = guard_info.append(row, ignore_index=True)   

In [ ]:
awake

### Question B